# Modern C++: management of dynamic resources

Scott Meyers: "A resource is something that, once you’re done using it, you need to return to the system. If you don’t, bad things happen. In C++ programs, the most commonly used resource is dynamically allocated memory (if you allocate memory and never deallocate it, you’ve got a memory leak), but memory is only one of many resources you must manage. Other common resources include file descriptors, mutex locks, fonts and brushes in graphical user interfaces (GUIs), database connections, and network sockets. Regardless of the resource, it’s important that it be released when you’re finished with it. Trying to ensure this by hand is difficult under any conditions, but when you consider exceptions, functions with multiple return paths, and maintenance programmers modifying software without fully comprehending the impact of their changes, it becomes clear that ad hoc ways of dealing with resource management aren’t sufficient."

## Motivation

Let's imagine a class library used for modeling of particles which is comprised of a base class, a factory (for example, random) and a function for printing.

In [ ]:
%%file tmp.raii.cpp

#include <iostream>
#include <string>
#include <random>

class Particle
 {
  public :
    virtual ~Particle() =default ;
    virtual std::string const & name() const =0 ;
 } ;
class Electron : public Particle
 {
  public :
    virtual std::string const & name() const override
      { static const std::string name{"Electron"} ; return name ; }
 } ;
class Proton : public Particle
 {
  public :
    virtual std::string const & name() const override
      { static const std::string name{"Proton"} ; return name ; }
 } ;
class Neutron : public Particle
 {
  public :
    virtual std::string const & name() const override
      { static const std::string name{"Neutron"} ; return name ; }
 } ;
 
Particle * newParticle()
 {
  std::random_device rd;
  std::mt19937 gen(rd());
  std::uniform_int_distribution<> dis(1,3);
  switch (dis(gen))
   {
    case 1: return new Electron{} ;
    case 2: return new Proton{} ;
    case 3: return new Neutron{} ;
   }
  return nullptr ;
 }

void printInfo( Particle const & p )
 { std::cout<<p.name()<<std::endl ; }

void f()
 {
  Particle * p = newParticle() ;
  //...
  printInfo(*p) ;
  //...
  delete p ;
 }

int main()
 {
  //...
  f() ;
  //...
 }

In [ ]:
!rm -f tmp.raii.exe && g++ -std=c++17 tmp.raii.cpp -o tmp.raii.exe

In [ ]:
!./tmp.raii.exe

A developer should not forget to delete all created particles once they are not used anymore. There can be multiple reasons preventing correct deletion of these objects:
* a premature call to `return` before the end of function is execution;
* in cases when a call to the factory and a call of the destructor are placed in a loop with calls `break`, `continue` or`goto`;
* an exception thrown before the end of function is execution.

If an object is not deleted, not only the memory allocated for it has not been freed, but also other resources. Even an experienced meticulous programmer can fall into this trap. For example if one of functions called in `f()` is "corrected" outside and starts raising exceptions.

## Solution: RAII (resource acquisition is initialization) idiom

In order to make sure that a resource is released at the end of its utilization, one can wrap this resource into a guard object whose destructor will release the resource. There is no more possible bypass because whenever one leaves a function, all local objects are always destructed.

```c++
//...
class ParticleGuard
 {
  public :
    ParticleGuard( Particle * p ) : m_p(p) {}
    ~ParticleGuard() { delete m_p ; }
  private :
    Particle * m_p ;
 } ;
//...
void f()
 {
  Particle * p = newParticle() ;
  ParticleGuard pg(p) ;
  //...
  printInfo(p) ;
  //...
 }
//...
```

Utilization of objects for resource management is based on two principles:
1. The allocated resource should be immediately confided to an resource-manager object. As frequently the resource-manager is initialized with the resource address, one talks about **RAII** (Resource Acquisition Is Initialization).
2. The manager objects use their destructors to release the resources. It does not matter which way the object scope is exited, the destructor will be called in any case.

 BEWARE: one should not create multiple guards for a single resource because otherwise this resource will be released several times.

```c++
//...
void f()
 {
  Particle * p = newParticle() ;
  ParticleGuard pg1(p) ;
  //...
  ParticleGuard pg2(p) ;
  //...
  printInfo(p) ;
  //...
 }
//...
```

To avoid this type of errors, the resource should be directly given to the guard... but we cannot access it anymore! Hence the need for some way to retrieve the resource from the guard:

```c++
//...
void f()
 {
  ParticleGuard pg(newParticle()) ;
  //...
  printInfo(???) ;
  //...
 }
//...
```

## Giving direct access to resources

If one wants to avoid duplication of the entire resource interface, direct access to the resource should be implemented.

#### Explicit conversion

While conceiving the concept of a guard class, one should foresee a method of direct access to a resource, for example, via a method `get`:

```c++
//...
class ParticleGuard
 {
  public:
    ParticleGuard( Particle * p ) : m_p{p} {}
    Particle & get() { return *m_p ; }
    ~ParticleGuard() { delete m_p ; }
  private:
    Particle * m_p ;
 } ;

void f()
 {
  ParticleGuard pg(newParticle()) ;
  //...
  printInfo(pg.get()) ;
  //...
 }
//...
```

#### Automatic conversion

Nevertheless, calling the function get() multiple times can become annoying to the point that a developer would prefer to bypass using a guard class. To avoid such a situation, it is a good idea to define a conversion operator that will be evoked automatically by the compiler whenever is needed.

```c++
//...
class ParticleGuard
 {
  public:
    ParticleGuard( Particle * p ) : m_p{p} {}
    operator Particle &() { return *m_p ; }
    ~ParticleGuard() { delete m_p ; }
  private:
    Particle * m_p ;
 } ;

void f()
 {
  ParticleGuard p(newParticle()) ;
  //...
  printInfo(p) ;
  //...
 }
//...
```

#### Some traps and remarks on class encapsulation

The possibility of automatic conversion opens the door to errors. In the below example a pointer is copied, not necessarily intentionally; if `p1` goes out of scope, `p2` turns into a dangling pointer.

```c++
ParticleGuard p1(newParticle()) ;
//...
Particle * p2 = p1 ;
//...
```

One can esteem that methods that give an access to a raw resource violate the idea of guard objects. It is not quite true: the guard class is not there to encapsulate the resource, but rather to ensure that it is freed correctly. If one wishes, it is always possible to encapsulate all into another layer.

## How to safely copy the guards?

We have already seen how to create objects `ParticleGuard`, which manage automatically the destruction of objects `Particle`. But what is going in case if a guard is copied? The answer is: it depends.

####  Basic strategy: forbid the copy

In many cases it does not make sense to copy a guard but the compiler provides a default copy constructor. We already know how to disable such a copy in C++11:

```c++
class ParticleGuard :
 {
  public :
    ...
    ParticleGuard( ParticleGuard const & ) =delete ;
    ParticleGuard & operator=( ParticleGuard const & ) =delete ;
    ...
 } ;
```

#### Robust (but complex) strategy: internally count the guards

On can introduce a counter of guards sharing the same resource. Once the counter is set to zero, the resource can be freed.

```c++
#include <iostream>

class ParticleGuard
 {
  public:
  
    ParticleGuard( Particle * p ) // acquire resource
     : m_bloc(new Particularization(p))
     {}                         

    ParticleGuard( ParticleGuard const & pg )
     : m_bloc(pg.m_bloc)
     { m_bloc->m_nb += 1 ; }                         

    ParticleGuard & operator=( ParticleGuard const & pg )
     {
      if (this==&pg) return *this ;
      m_bloc->m_nb -= 1 ;
      if (m_bloc->m_nb==0)
        delete m_bloc ;
      m_bloc = pg.m_bloc ;
      m_bloc->m_nb += 1 ;
      return *this ;
     }                         

    operator Particle &() { return *m_bloc->m_p ; }

    ~ParticleGuard() // release resource
     {
      m_bloc->m_nb -= 1 ;
      if (m_bloc->m_nb==0)
        delete m_bloc ;
     }                

  private :
  
    struct Particularization
     {
      Particularization( Particle * p ) : m_p(p) {}
      ~Particularization() { delete m_p ; }
      unsigned int m_nb{1} ;
      Particle * m_p ;
     } ;
  
    Particularization * m_bloc ;
    
 } ;

void f()
 {
  ParticleGuard pg1(newParticle()) ;
  //...
  ParticleGuard pg2(pg1) ;
  //...
 }
//...
```

#### Simple (but not universal) strategy: deep copy

Sometimes it is preferable to duplicate the resource each time its guard is copied. In our example there is an additional difficulty is that the real class of the pointed particle is not known, hence the need for some virtual `clone` function in the `Particle` inheritance tree

```c++
class Particle
 {
  //...
  virtual Particle * clone() const =0 ;
 } ;
class Electron : public Particle
 {
  //...
  virtual Particle * clone() const { return new Electron{*this} ; } ;
 } ;
//...
class ParticleGuard :
 {
  public :
    //...
    ParticleGuard( ParticleGuard const & pg )
     { m_p = pg.m_p->clone() ; }
    ParticleGuard & operator=( ParticleGuard const & pg )
     { delete m_p ; m_p = pg.m_p->clone() ; return *this ; }
    //...
 } ;
```

####  Subtle (but unnatural) strategy: transfer the resource

In rare cases we may want to allow a copy of a guard stealing the resource from the former guard and leaving it "empty".

```c++
class ParticleGuard :
 {
  public :
    //...
    ParticleGuard( ParticleGuard const & pg )
     { m_p = pg.m_p ; pg.m_p = nullptr ; }
    ParticleGuard & operator=( ParticleGuard const & pg )
     { delete m_p ; m_p = pg.m_p ; pg.m_p = nullptr ; }
    //...
 } ;
```

#### To remember

  - Copying a guard (RAII) raises questions about the copy of the resource itself. The resource copy policy dictates the guard copy strategy.
  - The most common RAII copying strategies are interdiction or counting references, but other alternatives are possible.

© *CNRS 2021*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*